# Restoring Color to Greyscale Images

In [22]:
import image
import os
import random
import numpy as np
import scipy as sp
import time

import lasagne
import theano
import theano.tensor as T

%matplotlib inline
from IPython.core.pylabtools import figsize
import matplotlib.pyplot as plt
from matplotlib import cm
figsize(16, 4)

In [23]:
import convnets
import train
reload(convnets)
reload(train)

SIZE = 100
IMDIR = "images/raw"

handles = [os.path.join(IMDIR, h) for h in os.listdir(IMDIR)]
print len(handles)

30113


In [24]:
base = convnets.LUMINOSITY_RATIO_NET
net, train_fn, val_fn = convnets.CreateTheanoExprs(
    base=base,
    height=SIZE,
    width=SIZE,
    learning_rate=0.01)
convnets.PrintNetworkShape(net)

DimshuffleLayer (None, 3, 100, 100) => (None, 100, 100, 3)
Conv2DLayer (None, 5, 100, 100) => (None, 3, 100, 100)
Conv2DLayer (None, 12, 100, 100) => (None, 5, 100, 100)
Conv2DLayer (None, 1, 100, 100) => (None, 12, 100, 100)
DimshuffleLayer (None, 100, 100) => (None, 1, 100, 100)
InputLayer  => (None, 100, 100)


In [25]:
batch_stats, val_stats, err, net  = train.Train(
    num_batches=10,
    validate_every_n_batches=4,
    height=SIZE,
    width=SIZE,
    batch_size=100,
    image_handles=handles,
    val_set_size=100,
    test_set_size=100,
    net=net,
    train_fn=train_fn,
    val_fn=val_fn)

Loading validation and testing images...
Starting training...
Training batch 0 of 100 images. Last time = 0.00 seconds. Last load time = 1.45 seconds. Last error = 1.00000.
Training batch 1 of 100 images. Last time = 2.72 seconds. Last load time = 1.42 seconds. Last error = 0.45137.
Training batch 2 of 100 images. Last time = 2.86 seconds. Last load time = 1.39 seconds. Last error = 0.43884.
Training batch 3 of 100 images. Last time = 2.73 seconds. Last load time = 1.30 seconds. Last error = 0.37570.

Validating...
Validated on 100 images in 1.15 seconds. Error = 0.31730.

Training batch 4 of 100 images. Last time = 2.76 seconds. Last load time = 1.31 seconds. Last error = 0.33414.
Training batch 5 of 100 images. Last time = 2.88 seconds. Last load time = 1.43 seconds. Last error = 0.31188.
Training batch 6 of 100 images. Last time = 2.75 seconds. Last load time = 1.31 seconds. Last error = 0.29434.
Training batch 7 of 100 images. Last time = 2.70 seconds. Last load time = 1.31 seconds

In [63]:
images = np.zeros((5000, SIZE, SIZE, 3))
train.LoadImages(handles, SIZE, SIZE, 5000, images)

5238

In [77]:
def ImStat(images):
    for i, color in enumerate(["  red", "green", " blue"]):
        channel = images[:, :, :, i]
        print "{c}: mean={m:.2f}, std={s:.2f}".format(
            c=color, m=channel.mean(), s=channel.std())
    grey = (images[:, :, :, 0] * 0.299 +
            images[:, :, :, 1] * 0.587 +
            images[:, :, :, 2] * 0.114)
    print " grey: mean={m:.2f}, std={s:.2f}".format(
        m=grey.mean(), s=grey.std())
    
    sh = images.shape
    rsh = (sh[0], sh[1] * sh[2], sh[3])
    ch = images.reshape(rsh)
    means = ch.mean(axis=1)
    stds = ch.std(axis=1)
    print "Mean of Means:", means.mean(axis=0)
    print " Std of Means:", means.std(axis=0)
    print " Mean of Stds:", stds.mean(axis=0)
    print "  Std of Stds:", stds.std(axis=0)

In [78]:
ImStat(images)

  red: mean=133.07, std=69.87
green: mean=139.37, std=67.78
 blue: mean=153.86, std=72.75
 grey: mean=139.14, std=66.13
Mean of Means: [ 133.06600034  139.36869996  153.8576187 ]
 Std of Means: [ 39.01780789  37.21569554  43.59037316]
 Mean of Stds: [ 55.45964996  54.28226949  55.17878639]
  Std of Stds: [ 16.85049745  16.21874785  18.63020146]


In [66]:
images.shape

(5000, 100, 100, 3)

In [ ]:
images.reshape((images.shape))